In [1]:
import sklearn
import math
import pandas
import numpy
from sklearn.linear_model import LinearRegression

In [2]:
df = pandas.DataFrame({"x": numpy.random.rand(100)})
df["y"] = -df["x"] * df["x"] + 4
df["z"] = df["y"] * 2 + df["x"]

In [3]:
from skopt.space import Real, Integer
from skopt.utils import use_named_args

space = [
    Integer(0, 1, name="intercept"),
    Real(0, 1, "uniform", name="dummy"),
    Real(1, 5, "uniform", name="scale"),    
]

In [4]:
from skopt import gp_minimize
from skopt import forest_minimize

class MLModel(object):
    def __init__(self, df, params={}):
        self.params = params
        self.df = df
    
    def set_params(self, **params):
        print(params)
        self.params = params

    def set_dataftame(self, df):
        self.df = df
        
    def opt_fit(self, space):
        #gp = gp_minimize(self.objective, space, n_calls=50, random_state=0)
        
        @use_named_args(space)
        def my_objective(**params):
            self.set_params(**params)
            obj = self._train()
            return -obj
            
        gp = forest_minimize(func=my_objective, dimensions=space)
        return gp
       
    def objective(self, **params):
        self.set_params(**params)
        obj = self._train()
        return obj
        
    def _train(self):
        df = self.df.copy()
        lm = LinearRegression(fit_intercept=self.params["intercept"])
        df["z"] = df["z"] + numpy.random.rand(len(df)) * self.params["scale"] + self.params["dummy"]
        lm.fit(df[["x", "y"]], df["z"])
        obj = lm.score(df[["x", "y"]], df["z"])
        return obj

In [5]:
from skopt import forest_minimize

In [6]:
ml = MLModel(df)

In [7]:
ml.set_params(intercept=0,dummy=0,scale=1)
ml._train()

{'intercept': 0, 'dummy': 0, 'scale': 1}


0.5803000280618191

In [8]:
ml.set_params(intercept=1,dummy=0,scale=1)
ml._train()

{'intercept': 1, 'dummy': 0, 'scale': 1}


0.6391151620273918

In [9]:
g = ml.opt_fit(space)

{'intercept': 0, 'dummy': 0.7148317322183356, 'scale': 2.9939886659508654}
{'intercept': 0, 'dummy': 0.12986770452523902, 'scale': 2.1285899779312683}
{'intercept': 0, 'dummy': 0.5890170765754695, 'scale': 1.1643103981251524}
{'intercept': 0, 'dummy': 0.28045126961797645, 'scale': 4.339768287418433}
{'intercept': 0, 'dummy': 0.1756990007410039, 'scale': 1.8011290449780695}
{'intercept': 1, 'dummy': 0.7361593391833776, 'scale': 3.76283481226753}
{'intercept': 1, 'dummy': 0.03232571003428542, 'scale': 1.0231474527820077}
{'intercept': 1, 'dummy': 0.6204761327236249, 'scale': 4.49666739903041}
{'intercept': 1, 'dummy': 0.3857186708138761, 'scale': 2.5060782872825165}
{'intercept': 0, 'dummy': 0.4330662696981289, 'scale': 3.506931887129465}
{'intercept': 1, 'dummy': 0.05286867248425266, 'scale': 1.4869858140628347}
{'intercept': 1, 'dummy': 0.37534494082046743, 'scale': 1.1085014462969593}
{'intercept': 1, 'dummy': 0.9751961183607238, 'scale': 1.1815877155369785}
{'intercept': 1, 'dummy': 

In [ ]:
g.fun

In [ ]:
g.x

In [ ]:
g

In [ ]:

from  sklearn.datasets  import load_boston
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

boston = load_boston()
X, y = boston.data, boston.target
n_features = X.shape[1]

# gradient boosted trees tend to do well on problems like this
reg = GradientBoostingRegressor(n_estimators=50, random_state=0)


In [ ]:
from skopt.space import Real, Integer
from skopt.utils import use_named_args


# The list of hyper-parameters we want to optimize. For each one we define the bounds,
# the corresponding scikit-learn parameter name, as well as how to sample values
# from that dimension (`'log-uniform'` for the learning rate)
space  = [Integer(1, 5, name='max_depth'),
          Real(10**-5, 10**0, "log-uniform", name='learning_rate'),
          Integer(1, n_features, name='max_features'),
          Integer(2, 100, name='min_samples_split'),
          Integer(1, 100, name='min_samples_leaf')]

# this decorator allows your objective function to receive a the parameters as
# keyword arguments. This is particularly convenient when you want to set scikit-learn
# estimator parameters
@use_named_args(space)
def objective(**params):
    reg.set_params(**params)

    return -np.mean(cross_val_score(reg, X, y, cv=5, n_jobs=-1,
                                    scoring="neg_mean_absolute_error"))

In [ ]:
import numpy as np
from skopt import gp_minimize
res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)

"Best score=%.4f" % res_gp.fun